## Optimizing Existing Networks

Nengo DL is not confined to opimizing cutom made networks, it can also be used to make existing networks better, or achieve the same result with fewer neurons. What this example will show is how to train a circular convolution network.

Circular convolution is a key operation used to process [semantic pointers](http://compneuro.uwaterloo.ca/research/spa/semantic-pointer-architecture.html). By optimizing this smaller network, larger more complex networks that utilize circular convolution can benefit.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import nengo
import nengo_dl
%load_ext nengo.ipynb
from nengo.spa import Vocabulary

import tensorflow as tf

To properly train the network, we generate novel training data by randomly generating semantic pointers.

In [ ]:
def gen_pointers(n_inputs, dims, seed):
    state = np.random.RandomState(seed)
    vocabulary = Vocabulary(dimensions=dims, rng=state, max_similarity=1)
    pairs = []
    for v in range(n_inputs):
        # keys start with A, second element starts with B, third starts with C
        conv_key = 'C' + str(v)
        point_key_1= 'A' + str(v)
        pointer_1 = vocabulary.create_pointer()
        point_key_2 = 'B' + str(v)
        pointer_2 = vocabulary.create_pointer()
        vocabulary.add(point_key_1, pointer_1)
        vocabulary.add(point_key_2, pointer_2)
        vocabulary.add(conv_key, vocabulary.parse(point_key_2 + "*" + point_key_1))
    
    A = np.asarray([vocabulary['A' + str(i)].v for i in range(n_inputs)])[:, None, :]
    B = np.asarray([vocabulary['B' + str(i)].v for i in range(n_inputs)])[:, None, :]
    C = np.asarray([vocabulary['C' + str(i)].v for i in range(n_inputs)])[:, None, :]
    return A, B, C, vocabulary


dimensions = 50
seed = 0

test_a, test_b, test_c, vocab = gen_pointers(10, dimensions, seed)

We want our optimized network to work with spiking LIF neurons, so we will use SoftLIFRate neurons (a differentiable approximation of LIF neurons) to train the network.

We'll start with the `nengo.networks.CircularConvolution` network, where all the parameters are initialized using the standard Nengo methods, and then further optimize those parameters using deep learning training methods.

In this example only 5 neurons are used per dimension for the circular convolution. This is fewer than would typically be used in a Nengo model, but the enhanced performance enabled by the training process will allow the network to function well with this restricted number of neurons.

In [ ]:
with nengo.Network(seed=seed) as net:
    net.config[nengo.Ensemble].neuron_type = nengo_dl.SoftLIFRate(sigma=0.1)
    net.config[nengo.Connection].synapse = None

    # Get the raw vectors for the pointers using `vocab['A'].v`
    a = nengo.Node(output=vocab['A0'].v)
    b = nengo.Node(output=vocab['B0'].v)

    # Make the circular convolution network with 5 neurons per dimension
    cconv = nengo.networks.CircularConvolution(5, dimensions=dimensions)

    # Connect the input nodes to the input slots `A` and `B` on the network
    nengo.Connection(a, cconv.input_a)
    nengo.Connection(b, cconv.input_b)

    # Probe the output
    out = nengo.Probe(cconv.output)
    out_filtered = nengo.Probe(cconv.output, synapse=0.01)

We now run the network in its default state to get an idea of the baseline performance. Ideally the output would be clearly `C0`, the result of the convolution between `A0` and `B0`, but we can see that it is poorly differentiated.

In [ ]:
with nengo.Simulator(net) as sim:
    sim.run(0.3)
plt.figure()
output_vocab = vocab.create_subset(["C%d" % i for i in range(10)])
plt.plot(sim.trange(), nengo.spa.similarity(sim.data[out_filtered], output_vocab))
plt.legend(output_vocab.keys, loc=4)
plt.ylim([-1, 1])
plt.xlabel("t [s]")
plt.ylabel("dot product");

Now we can optimize our network, by showing it random input pointers and training it to output their circular convolution.

In [ ]:
with nengo_dl.Simulator(net, minibatch_size=100, device="/cpu:0") as sim:
    optimizer = tf.train.RMSPropOptimizer(5e-3)
        
    # generate random data
    train_a, train_b, train_c, _ = gen_pointers(1000, dimensions, seed+1)
    input_feed = {a: train_a, b: train_b}
    output_feed = {out: train_c}

    # train the network for one epoch
    sim.train(input_feed, output_feed, optimizer, n_epochs=100, objective="mse")

    sim.run(0.3)

After training we run the same test on the network and plot the output. Now we can clearly see that the output of the network is closest to the ideal output, `C0`.

In [ ]:
output = sim.data[out_filtered]
plt.figure()
plt.plot(sim.trange(), nengo.spa.similarity(output[0], output_vocab))
plt.legend(output_vocab.keys, loc=4)
plt.ylim([-1, 1])
plt.xlabel("t [s]")
plt.ylabel("dot product");

In a future example we will show how to integrate these training improvments into a larger network and improve the performance of the network as a whole.